# **Lab3: Linear regression with Sklearn and Statmodels**

## Reproducing the results from section 3.1 and 3.2.1 of _Intro to Stat Learning_

**WHAT** In this nonmandatory lab, we will look at linear regression and multiple regression for the Advertisement dataset.  We will follow the methods described in section 3.1 and 3.2.1 from _An introduction to Statistical Learning_.  We will find that `sklearn` is not equipped to produce standard errors for estimators; we'll do some "by hand" and for the multiple regression use `statsmodels`.

**WHY** The exercises are meant to familiarize yourself with the linear regression tools, using two different libraries.

**HOW** Follow the exercises in this notebook either on your own or with a fellow student.  Work your way through these exercises at your own pace and be sure to ask questions to the TA's when you don't understand something.

$\newcommand{\q}[1]{\rightarrow \textbf{Question #1.}}$
$\newcommand{\ex}[1]{\rightarrow \textbf{Exercise #1.}}$

## 1. Advertising Data

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
Advertising = pd.read_csv("Advertising.csv")
print("Advertising.shape =",Advertising.shape)
Advertising.head()

## 2. Simple linear regression, for each of the three predictors in turn

$\ex{1}$ The code below plots for each feature a scatterplot of its value against the output (Sales). Complete the code by creating a LinearRegression model from `sklearn.linear_model.LinearRegression` and adding the regression line to each of the plots. The resulting plots are the ones from Figure 2.1 in _ISL_.

Also store the residuals $e_i = y_i - \hat{y}_i, i=1, \ldots, n$ for TV versus Sales as TV_residuals. 

In [ ]:
from sklearn.linear_model import LinearRegression

data = np.array(Advertising)
column_names = Advertising.columns
TV_residuals = [] 

for i in range(3):
    fig, ax = plt.subplots()
    X_train = data[: , i].reshape(-1,1) # Training set samples for particular feature 
    y_train = data[: , 3] # Training set sales values 
    plt.scatter(X_train , y_train, color='red', marker = '.') #Scatter plot 
    plt.xlabel(column_names[i])
    plt.ylabel("sales")
    
    # START ANSWER 
    lr = LinearRegression() 
    lr.fit(X_train, y_train)
    
    intcpt, slope = lr.intercept_, lr.coef_
    xlim = ax.get_xlim()
    ylim = [slope * xlim[0] + intcpt, slope * xlim[1] + intcpt]
    ax.plot(xlim, ylim, linewidth = 3)
    plt.title('Regression fit of sales onto ' + column_names[i])
    
    if i == 0:
        predictions = lr.predict(X_train)
        TV_residuals = y_train - predictions
    # END ANSWER
    
    plt.show()

Recall the residual sum of squares (RSS) for simple linear regression: 
$$\text{RSS} = e_1^2 + e_2^2 + \ldots + e_n^2 = 
(y_1 - \beta_0 -\beta_1 x_1)^2 + \cdots + (y_n - \beta_0 -\beta_1 x_n)^2.$$

This is minimized over $(\beta_0, \beta_1)$ to obtain the least squares estimates.  We now make a contourplot of the RSS near the minimum, for the first model (TV), as in _ISL_ Figure 3.2.

In [ ]:
b_0s = np.linspace(5, 9, 100)
b_1s = np.linspace(0.03, 0.06, 100)
X_train = data[: , 0]

# Initialize an array to store RSS values
rss_values = np.zeros((len(b_0s), len(b_1s)))

# Calculate RSS for each combination of B_0 and B_1
for i, b_0 in enumerate(b_0s):
    for j, b_1 in enumerate(b_1s):
        y_pred = b_0 + b_1 * X_train
        rss_values[i, j] = np.sum(np.square(y_train - y_pred)) /1000

# Create a contour plot
contour = plt.contour(b_0s, b_1s, rss_values, 30)  # levels control the number of contour lines
plt.colorbar(contour)  # Adding a colorbar
plt.xlabel('$\\beta_0$')
plt.ylabel('$\\beta_1$')
plt.title('Contour plot of RSS as function of $\\beta_0$ and $\\beta_1$')

plt.show()

$\ex{2}$ Let us have a closer look at the TV fit; plot the residuals that you saved earlier in a scatter plot.

In [ ]:
# START ANSWER 
TV_points = data[: , 0] 
plt.scatter(TV_points , TV_residuals)
plt.xlabel("TV")
plt.ylabel("residuals")
plt.show()
# END ANSWER

$\q{3}$ In *Introduction to Statistical Learning*, a bit after $Y = \beta_0 + \beta_1 X + \epsilon$ (3.5), it says: 
> *We typically assume that the error term is independent of X.*

Looking at the plot, do you think this justified for the TV-only model? Why (not)?

<div style="background-color:#ef8b22">
    
Write your answer here:

[//]: # (START ANSWER)
The residuals seem to get larger for larger values of the TV feature, so it seems that ${\rm Var}(\epsilon)$ *does depend* on the predictor variable. This is called *heteroscedasticity*. What we want is *homoscedasticity*. So: no, it does not seem justified.

[//]: # (END ANSWER)

## 3. Assessing the accuracy of the coefficient estimates 
Before we actually move on to computing standard errors for our estimates, we are going to do a small simulation to get an idea where the uncertainty in the estimates comes from.

$\ex{3}$ The code below plots the population regression line and the least squares estimates. Furthermore, ten least
squares lines are shown, each computed on the basis of a separate random set of observations, as in _ISL_ Figure 3.3. 

In [ ]:
rng = np.random.default_rng(8701659)
n = 100
xsig = 0.9
x = rng.normal(0,xsig, n)  # normally distributed x-values
sigma = 4
y_true = 3 * x + 2

# formula's for the standard errors (see ISL 3.1.2)
xmean = np.mean(x)
xvar = np.var(x)
se_b0 = sigma * np.sqrt( (1/n) + xmean**2 / (n * xvar) )
se_b1 = sigma / np.sqrt(  n * xvar )
print(f"The SEs for beta_0 and beta_1: {se_b0:.3f} and {se_b1:.3f}") 

fig, ax = plt.subplots(1,2, figsize=(10, 5))

epsilon = rng.normal(0, sigma, n)
y1 = y_true + epsilon
lr = LinearRegression() 
lr.fit(x.reshape(-1,1),y1)

ax[0].scatter(x, y1,  color='black', edgecolor='black', facecolor='none')
ax[0].plot(x, y_true, color = 'red', label = "population regression line")
ax[0].plot(x, lr.predict(x.reshape(-1,1)), label = f"least squares line", color = "blue") 

# extra runs
for i in range(10):
    epsilon = rng.normal(0, sigma, n)
    y = y_true + epsilon
    lr = LinearRegression() 
    lr.fit(x.reshape(-1,1), y)
    ax[1].plot(x, lr.predict(x.reshape(-1,1)), color = 'lightblue', linewidth = 0.5) 
    print(f"Regression line {i}: beta_0= {lr.intercept_:.3f}, beta_1= {lr.coef_[0]:.3f}")

lr.fit(x.reshape(-1,1),y1)
ax[1].plot(x, y_true, color = 'red', label = "population regression line")
ax[1].plot(x, lr.predict(x.reshape(-1,1)), label = f"least squares line", color = "blue")

ax[0].set_ylim(-11, 13)
ax[1].set_ylim(-11, 13)
ax[0].set_xlim(-2.1, 2.3)
ax[1].set_xlim(-2.1, 2.3)
ax[0].legend()
ax[1].legend()

You can experiment by playing with the parameters `sigma`, `xsig` (controls the spread of the x's), `n` and observe the effects: what happens if you double $\sigma$, or halve `xsig`? Do you understand what you see?

## 4. Advertising data: Multiple Linear Regression

## With Sklearn
Using Sklearn's `LinearRegression` we have to compute our own standard errors, but as you can see here, this is rather cumbersome and prone to errors.

In [ ]:
# Single features 
for i in range(3):
    lr = LinearRegression() 
    lr.fit(data[:, i].reshape(-1,1) , data[:, 3].reshape(-1,1)) 
    coefficient = lr.coef_ 
    intercept = lr.intercept_
    mean = np.mean(data[:, i])     
    y_pred = lr.predict(data[:, i].reshape(-1,1))
    residuals = np.sum(np.square(data[:, 3].reshape(-1,1) - y_pred)) 
    sigma_2 = residuals / 198
    TSS = np.sum(np.square(data[:, 3] - np.mean(data[:, 3])))
    standard_error_b0 = np.sqrt((sigma_2 * ((1 / len(data)) + np.square(mean) / (np.sum(np.square(data[:, i] - mean))))))
    standard_error_b1 = np.sqrt(sigma_2 / np.sum(np.square(data[:, i] - mean)))
    t_statistic_coefficient =  coefficient[0][0] / standard_error_b0
    print(f"coefficient of {column_names[i]} is {coefficient[0][0].round(4)} with SE {standard_error_b1.round(4)} and intercept is {intercept[0].round(4)} with SE {standard_error_b0.round(4)}")
    print(f"It has an RSE of {np.sqrt(residuals / 198).round(3)} and an R^2 of {(1 - residuals/TSS).round(3)} ")
    
print()

# Multiple features     
lr = LinearRegression() 
lr.fit(data[:, :3] , data[:, 3].reshape(-1,1)) 
coefficients = lr.coef_
intercept = lr.intercept_
y_pred = lr.predict(data[:, :3])
residuals = np.sum(np.square(data[:, 3].reshape(-1,1) - y_pred)) 
sigma_2 = residuals / 198
for i in range(3):
    mean = np.mean(data[:, i])
    standard_error_b1 = np.sqrt(sigma_2 / np.sum(np.square(data[:, i] - mean)))
    t_statistic = coefficients[0][i] / standard_error_b1
    print(f"coefficient of  {column_names[i]} is {coefficients[0][i].round(4)} with SE of {standard_error_b1.round(4)} and t-statistic {t_statistic.round(4)}")

## With Statsmodels
Statsmodels has methods that have builtin the determination of the statistics we are after.

In [ ]:
# new imports
import statsmodels.api as sm

$\ex{4}$ Using `Statsmodels`'s `OLS`, fit the models using 
>1. the features independently 
2. all the features together

to compute feature coefficients, standard errors, t-statistics, residual standard errors and $R^2$. You will need to build an OLS model and then `fit()` will produce a `RegressionResults` object with all you need. In the documentation, look for methods with "summary". 

In [ ]:
# TV
# START ANSWER 
y = Advertising['sales']
X = pd.DataFrame({'intercept': np.ones(Advertising.shape[0]), 'TV': Advertising['TV']})
model = sm.OLS(y, X)
results = model.fit()

# fig, ax = subplots()
ax =  Advertising.plot.scatter('TV', 'sales', color = 'red', marker = '.')
intcpt, slope = results.params
xlim = ax.get_xlim()
ylim = [slope * xlim[0] + intcpt, slope * xlim[1] + intcpt]
ax.plot(xlim, ylim, linewidth = 3)

results.summary2()
# END ANSWER

In [ ]:
# radio
# START ANSWER
y = Advertising['sales']
X = sm.add_constant(Advertising['radio'])
model = sm.OLS(y, X)
results = model.fit()
results.summary()
# END ANSWER

In [ ]:
# newspaper
# START ANSWER
y = Advertising['sales']
X = sm.add_constant(Advertising['newspaper'])
model = sm.OLS(y, X)
results = model.fit()
results.summary2()
# END ANSWER

In [ ]:
# all three predictors
# START ANSWER 
y = Advertising['sales']
X = sm.add_constant(Advertising.iloc[:,:3])
model = sm.OLS(y, X)
results = model.fit()
results.summary2()
# END ANSWER

$\ex{5} Compute the covariance matrix and check it against the